<a href="https://colab.research.google.com/github/JazmineOrtizMarin/Simulaci-n-2/blob/main/MC_Tiempo_AyC_vs_CRD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import random as rd
import timeit
import time

In [27]:
def g(x):
  return np.sqrt(np.arctan(x))

**Monte Carlo: Acierto y Error**



In [28]:
def estimacion_1(n):
  aciertos=0
  for i in range (n):
    x = rd.random()
    y = rd.random()

    if y <= g(x):
      aciertos+=1
  I=aciertos/n
  return I

In [29]:
estimacion_1(1000)

0.64

Hacemos una función que la llame 50 veces, de ahí calcular la varianza y medimos el tiempo:

In [42]:
def evaluar_acierto_y_error(n_puntos, repeticiones=50):
  inicio_tiempo = time.time()
  lista_resultados = [] #Arreglo para guardar los resultados
  for _ in range(repeticiones):
    resultado = estimacion_1(n_puntos)
    lista_resultados.append(resultado)
  fin_tiempo = time.time()
  tiempo_total = fin_tiempo - inicio_tiempo
  varianza_resultados = np.var(lista_resultados)
  return tiempo_total, varianza_resultados

In [43]:
n = 1000
t_1, var_1 = evaluar_acierto_y_error(n, repeticiones=50)

print(f"Acierto y Error (50 ejecuciones):", "\n-------------------------------------")
print(f"Tiempo de ejecución (t_1): {t_1:.5f}s")
print(f"Varianza (var_Theta_1): {var_1:.8f}")

Acierto y Error (50 ejecuciones): 
-------------------------------------
Tiempo de ejecución (t_1): 0.20181s
Varianza (var_Theta_1): 0.00015995


**Monte Carlo: Crudo**

In [39]:
def crudo(N):
  G=[]
  for i in range (N):
    u=rd.random()
    G.append(g(u))
  return np.mean(G), np.var(G), np.std(G), G
#e=crudo(1000)
#print("El valor estimado es: ", e[0], "\nVarianza: ", e[1])

In [40]:
e=crudo(1000)

Llamamos a la función 50 veces para calcular el tiempo:



In [41]:
def evaluar_crudo(n_puntos, repeticiones=50):
  inicio_tiempo = time.time()
  lista_de_medias = []
  for _ in range(repeticiones):
    resultado_completo = crudo(n_puntos)
    estimacion_integral = resultado_completo[0]
    lista_de_medias.append(estimacion_integral)
  fin_tiempo = time.time()
  tiempo_total = fin_tiempo - inicio_tiempo
  varianza_de_medias = np.var(lista_de_medias)
  return tiempo_total, varianza_de_medias

In [44]:
n = 1000
t_2, var_2 = evaluar_crudo(n, repeticiones=50)

print(f"Resultados para el método Crudo (50 ejecuciones):")
print("--------------------------------------------------")
print(f"Tiempo total de ejecución (t_2): {t_2:.5f} segundos")
print(f"Varianza de los resultados (var_2): {var_2:.8f}")

Resultados para el método Crudo (50 ejecuciones):
--------------------------------------------------
Tiempo total de ejecución (t_2): 0.11733 segundos
Varianza de los resultados (var_2): 0.00003902


**EFICIENCIA**